![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Lab 9: InSAR Time Series Analysis using GIAnT within Jupyter Notebooks

### Franz J Meyer & Joshua J C Knicely; University of Alaska Fairbanks

<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />

This Lab is part of the UAF course [GEOS 657: Microwave Remote Sensing](https://radar.community.uaf.edu/). The primary goal of this lab is to demonstrate how to process InSAR data, specifically interferograms, using the Generic InSAR Analysis Toolbox [GIAnT](http://earthdef.caltech.edu/projects/giant/wiki) in the framework of *Jupyter Notebooks*.

**Our specific objectives for this lab are to:**

- Learn how to prepare data for GIAnT. 
- Use GIAnT to create maps of surface deformation. 
    -  Understand its capabilities. 
    -  Understand its limitations. 

## Target Description

In this lab, we will analyze the volcano Sierra Negra. This is a highly active volcano on the Galapagos hotpsot. The most recent eruption occurred from 29 June to 23 August 2018. The previous eruption occurred in October 2005, prior to the launch of the Sentinel-1 satellites, which will be the source of data we use for this lab. We will be looking at the deformation that occurred prior to the volcano's 2018 eruption.

In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## Overview

**About GIAnT**

GIAnT is a Python framework that allows rapid time series analysis of low amplitude deformation signals. It allows users to use multiple time series analysis technqiues: Small Baseline Subset (SBAS), New Small Baseline Subset (N-SBAS), and Multiscale InSAR Time-Series (MInTS). As a part of this, it includes the ability to correct for atmospheric delays by assuming a spatially uniform stratified atmosphere. 

**Limitations**

GIAnT has a number of limitations that are important to keep in mind as these can affect its effectiveness for certain applications. It implements the simplest time-series inversion methods. Its single coherence threshold is very conservative in terms of pixel selection. It does not include any consistency checks for unwrapping errors. It has a limited dictionary of temporal model functions. It cannot correct for atmospheric effects due to differing surface elevations. 

**Steps to use GIAnT**
Although GIAnT is an incredibly powerful tool, it requires very specific input. Because of the input requirements, the majority of one's effort goes to getting the data into a form that GIAnT can manipulate and to creating files that tell GIAnT what to do. The general steps to use GIAnT are below. 

- Download Data
- Identify Area of Interest
- Subset (Crop) Data to Area of Interest
- Prepare Data for GIAnT
    - Adjust file names
    - Remove potentially disruptive default values (optional)
    - Convert data from '.tiff' to '.flt' format
- Create Input Files for GIAnT
    - Create 'ifg.list'
    - Create 'date.mli.par'
    - Make prepxml_SBAS.py
    - Run prepxml_SBAS.py
    - Make userfn.py
- Run GIAnT
    - PrepIgramStack.py*
    - ProcessStack.py
    - SBASInvert.py
    - SBASxval.py
- Data Visualization

The steps from PrepIgramStack.py and above have been completed for you in order to save disk space and computation time. This allows us to concentrate on the usage of GIAnT and data visualization. Some of the code to create the prepatory files (e.g., 'ifg.list', 'date.mli.par', etc.) have been incldued for your potential use. More information about GIAnT can be found [here](http://earthdef.caltech.edu/projects/giant/wiki).

**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

## 0. Import Python Libraries:

**Import the Python libraries and modules we will need to run this lab:**

In [ ]:
%%capture
from datetime import date
from pathlib import Path

import h5py # for is_hdf5
import os # need os for context switch
import shutil

from osgeo import gdal
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib import rc
import numpy as np

from IPython.display import HTML

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

## 1. Transfer data to a local directory

The data cube (referred to as a stack in the GIAnT documentation and code) and several other needed files have been created and stored in the GIAnT server. We will download this data to a local directory and unzip it.

Before we download anything, **create a working directory for this analysis and change into it:**

In [ ]:
work_path = Path.cwd()/'lab_9_data'
if not work_path.exists():
    work_path.mkdir()

First step is to find the zip file and download it to a local directory. This zip file has been placed in the S3 bucket for this class.

**Display the contents of the S3 bucket:**

In [ ]:
!aws s3 ls --region=us-east-1 --no-sign-request s3://asf-jupyter-data/

**Copy the desired file ('Lab9Files.zip') to your data directory:**

In [ ]:
!aws s3 cp --region=us-east-1 --no-sign-request s3://asf-jupyter-data/Lab9Files.zip $work_path/Lab9Files.zip

**Create the directories where we will perform the GIAnT analysis and store the data:**

In [ ]:
stack_path = work_path/'Stack'

if not stack_path.exists():
    stack_path.mkdir()

**Extract the zipped file to path and delete it:**

In [ ]:
zipped_path = work_path/'Lab9Files.zip'
asfn.asf_unzip(str(work_path), str(zipped_path))

if zipped_path.is_file:
    zipped_path.unlink()

The files have been extracted and placed in a folder called 'Lab9Files'. 

**Move the amplitude image, data.xml, date.mli.par, and sbas.xml files to path and RAW-STACK.h5 to stack_path:**

In [ ]:
print(type(temp_dir))

In [ ]:
temp_dir = work_path/'Lab9Files'

# Move 'RAW-STACK.h5' to 'Stack' directory
if not Path(f'{stack_path}/RAW-STACK.h5').exists():
    shutil.move(f'{temp_dir}/RAW-STACK.h5', stack_path)
    (temp_dir/'RAW-STACK.h5').unlink()

# Move rest of content into directory above
files = list(temp_dir.rglob("*.*"))
for file in files:
    if file.exists():
        try:
            file.move(work_path/f"{file.name}")
        except:
            file.unlink()
        
# Remove empty 'Lab9Files' directory
if temp_dir.exists():
    temp_dir.rmdir()

## 2. Create Input Files And Code for GIAnT

The code below shows how to create the input files and specialty code that GIAnT requires. For this lab, 'ifg.list' is not needed, 'date.mli.par' has already been provided, 'prepxml_SBAS.py' is not needed as the 'sbas.xml' and 'data.xml' files it would create have already been provided, and 'userfn.py' is not needed as we are skipping the step in which it would be used. <br>The files that would be created are listed below. 
   
- ifg.list
    - List of the interferogram properties including master and slave date, perpendicular baseline, and sensor. 
- date.mli.par
    - File from which GIAnT pulls requisite information about the sensor. 
    - This is specifically for GAMMA files. When using other interferogram processing techniques, an alternate file is required. 
- prepxml_SBAS.py
    - Python function to create an xml file that specifies the processing options to GIAnT. 
    - This must be modified by the user for their particular application. 
- userfn.py
    - Python function to map the interferogram dates to a phyiscal file on disk. 
    - This must be modified by the user for their particular application. 
    
### 2.1 Create 'ifg.list' File

This will create simple 4 column text file will communicate network information to GIAnT. It will be created within the **GIAnT** folder.

**This step has already been done, so we will not actually create the 'ifg.list' file. This code is displayed for your potential future use.**

In [ ]:
"""
# Get one of each file name. This assumes the unwrapped phase geotiff has been converted to a '.flt' file
files = [f for f in os.listdir(datadirectory) if f.endswith('_unw_phase.flt')] 

# Get all of the master and slave dates. 
masterDates,slaveDates = [],[]
for file in files:
    masterDates.append(file[0:8])
    slaveDates.append(file[9:17])
# Sort the dates according to the master dates. 
master_dates,sDates = (list(t) for t in zip(*sorted(zip(masterDates,slaveDates))))

with open( os.path.join('GIAnT', 'ifg.list'), 'w') as fid:
    for i in range(len(master_dates)):
        masterDate = master_dates[i] # pull out master Date (first set of numbers)
        slaveDate = sDates[i] # pull out slave Date (second set of numbers)
        bperp = '0.0' # according to JPL notebooks
        sensor = 'S1' # according to JPL notebooks
        fid.write(f'{masterDate}  {slaveDate}  {bperp}  {sensor}\n') # write values to the 'ifg.list' file. 
"""

You may notice that the code above sets the perpendicular baseline to a value of 0.0 m. This is not the true perpendicular baseline. That value can be found in metadata file (titled '$<$master timestamp$>$_$<$slave timestamp$>$.txt') that comes with the original interferogram. Generally, we would want the true baseline for each interferogram. However, since Sentinel-1 has such a short baseline, a value of 0.0 m is sufficient for our purposes.

### 2.2 Create 'date.mli.par' File

As we are using GAMMA products, we must create a 'date.mli.par' file from which GIAnT will pull necessary information. If another processing technique is used to create the interferograms, an alterante file name and file inputs are required.

**Again, this step has already been completed and the code is only displayed for your potential future use.**

In [ ]:
"""
# Create file 'date.mli.par'

# Get file names
files = [f for f in os.listdir(datadirectory) if f.endswith('_unw_phase.flt')]

# Get WIDTH (xsize) and FILE_LENGTH (ysize) information
ds = gdal.Open(datadirectory+files[0], gdal.GA_ReadOnly)
type(ds)

nLines = ds.RasterYSize
nPixels = ds.RasterXSize

trans = ds.GetGeoTransform()
ds = None

# Get the center line UTC time stamp; can also be found inside <date>_<date>.txt file and hard coded
dirName = os.listdir('ingrams')[0] # get original file name (any file can be used; the timestamps are different by a few seconds)
vals = dirName.split('-') # break file name into parts using the separator '-'
tstamp = vals[2][9:16] # extract the time stamp from the 2nd datetime (could be the first)
c_l_utc = int(tstamp[0:2])*3600 + int(tstamp[2:4])*60 + int(tstamp[4:6])

rfreq = 299792548.0 / 0.055465763 # radar frequency; speed of light divided by radar wavelength of Sentinel1 in meters

# write the 'date.mli.par' file
with open(os.path.join(path, 'date.mli.par'), 'w') as fid:
    # Method 1
    fid.write(f'radar_frequency: {rfreq} \n') # when using GAMMA products, GIAnT requires the radar frequency. Everything else is in wavelength (m) 
    fid.write(f'center_time: {c_l_utc} \n') # Method from Tom Logan's prepGIAnT code; can also be found inside <date>_<date>.txt file and hard coded
    fid.write( 'heading: -11.9617913 \n') # inside <date>_<date>.txt file; can be hardcoded or set up so code finds it. 
    fid.write(f'azimuth_lines: {nLines} \n') # number of lines in direction of the satellite's flight path
    fid.write(f'range_samples: {nPixels} \n') # number of pixels in direction perpendicular to satellite's flight path
    fid.close() # close the file
"""

### 2.3 Make prepxml_SBAS.py

We will create a prepxml_SBAS.py function and put it into our GIAnT working directory. Again, this is shown for anyone that may want to use GIAnT on their own.

If we do wish to change `sbas.xml` or `data.xml`, this can be done by creating and running a new 'prepxml_SBAS.py'.

#### 2.3.1 Necessary prepxml_SBAS.py edits

GIAnT comes with an example prepxml_SBAS.py, but requries significant edits for our purposes. These alterations have already been made, so we don't have to do anything now, but it is good to know the kinds of things that have to be altered. The details of some of these options can be found in the GIAnT documentation. The rest must be found in the GIAnT processing files themselves, most notably the tsxml.py and tsio.py functions.

**The following alterations were made:**

- Changed 'example' &#9658; 'date.mli.par'
- Removed 'xlim', 'ylim', 'ref_x_lim', and 'ref_y_lim'
    - These are used for clipping the files in GIAnT. As we have already done this, it is not necessary. 
- Removed latfile='lat.map' and lonfile='lon.map'
    - These are optional inputs for the latitude and longitude maps. 
- Removed hgtfile='hgt.map'
    - This is an optional altitude file for the sensor. 
- Removed inc=21.
    - This is the optional incidence angle information. 
    - It can be a constant float value or incidence angle file. 
    - For Sentinel1, it varies from 29.1-46.0&deg;.
- Removed masktype='f4'
    - This is the mask designation. 
    - We are not using any masks for this. 
- Changed unwfmt='RMG' &#9658; unwfmt='GRD'
    - Read data using GDAL. 
- Removed demfmt='RMG'
- Changed corfmt='RMG' &#9658; corfmt='GRD'
    - Read data using GDAL. 
- Changed nvalid=30 -> nvalid=1
    - This is the minimum number of interferograms in which a pixel must be coherent. A particular pixel will be included only if its coherence is above the coherence threshold, cohth, in more than nvalid number of interferograms. 
- Removed atmos='ECMWF'
    - This is an amtospheric correction command. It depends on a library called 'pyaps' developed for GIAnT. This library has not been installed yet. 
- Changed masterdate='19920604' &#9658; masterdate='20161119'
    - Use our actual masterdate. 
    - I simply selected the earliest date as the masterdate. 
    
---

Defining a reference region is a potentially important step. This is a region at which there should be no deformation. For a volcano, this should be some significant distance away from the volcano. GIAnT has the ability to automatically select a reference region which we will use for this exercise. 

Below is an example of how the reference region would be defined. If we look at the prepxml_SBAS.py code below, ref_x_lim and ref_y_lim, the pixel based location of the reference region, is within the code, but has been commented out. 

**Define reference region:**

In [ ]:
ref_x_lim, ref_y_lim = [0, 10], [95, 105]

Below is an example of how the reference region would be defined. Look at the prepxml_SBAS.py code below. Note that ref_x_lim and ref_y_lim (the pixel based location of the reference region) are within the code.

**This has already been completed but the code is here as an example script for creating XML files for use with the SBAS processing chain.**

In [ ]:
'''
#!/usr/bin/env python

import tsinsar as ts
import argparse
import numpy as np

def parse():
    parser= argparse.ArgumentParser(description='Preparation of XML files for setting up the processing chain. Check tsinsar/tsxml.py for details on the parameters.')
    parser.parse_args()

parse()
g = ts.TSXML('data')
g.prepare_data_xml(
    'date.mli.par', proc='GAMMA', 
    #ref_x_lim = [{1},{2}], ref_y_lim=[{3},{4}],
    inc = 21., cohth=0.10, 
    unwfmt='GRD', corfmt='GRD', chgendian='True', endianlist=['UNW','COR'])
g.writexml('data.xml')


g = ts.TSXML('params')
g.prepare_sbas_xml(nvalid=1, netramp=True, demerr=False, uwcheck=False, regu=True, masterdate='{5}', filt=1.0)
g.writexml('sbas.xml')


############################################################
# Program is part of GIAnT v1.0                            #
# Copyright 2012, by the California Institute of Technology#
# Contact: earthdef@gps.caltech.edu                        #
############################################################

'''

**Set the master date and create a script for creating XML files for use with the SBAS processing chain:**

In [ ]:
#files = [f for f in os.listdir(datadirectory) if f.endswith('_unw_phase.flt')]
#master_date = min([files[i][0:8] for i in range(len(files))], key=int)

master_date = '20161119'

prepxml_SBAS_Template = '''
#!/usr/bin/env python
"""Example script for creating XML files for use with the SBAS processing chain. This script is supposed to be copied to the working directory and modified as needed."""


import tsinsar as ts
import argparse
import numpy as np

def parse():
    parser= argparse.ArgumentParser(description='Preparation of XML files for setting up the processing chain. Check tsinsar/tsxml.py for details on the parameters.')
    parser.parse_args()

parse()
g = ts.TSXML('data')
g.prepare_data_xml(
    'date.mli.par', proc='GAMMA', 
    #ref_x_lim = [{1},{2}], ref_y_lim=[{3},{4}],
    inc = 21., cohth=0.10, 
    unwfmt='GRD', corfmt='GRD', chgendian='True', endianlist=['UNW','COR'])
g.writexml('data.xml')


g = ts.TSXML('params')
g.prepare_sbas_xml(nvalid=1, netramp=True, demerr=False, uwcheck=False, regu=True, masterdate='{5}', filt=1.0)
g.writexml('sbas.xml')


############################################################
# Program is part of GIAnT v1.0                            #
# Copyright 2012, by the California Institute of Technology#
# Contact: earthdef@gps.caltech.edu                        #
############################################################

'''

# with open(work_path.joinpath('prepxml_SBAS.py'), 'w') as fid:
with open(work_path/'prepxml_SBAS.py', 'w') as fid:
    fid.write(prepxml_SBAS_Template.format(work_path,ref_x_lim[0],ref_x_lim[1],ref_y_lim[0],ref_y_lim[1],master_date))

To create a new 'sbas.xml' and 'data.xml' file, we would modify the above code to give new parameters and to write to the appropriate folder (e.g., to change the time filter from 1 year to none and to write to the directory in which we are working; 'filt=1.0' -> 'filt=0.0'; and 'os.path.join(path,'prepxml_SBAS.py') -> 'prepxml_SBAS.py' OR '%cd ~' into your home directory). Then we would run it below.

### 2.4 Run prepxml_SBAS.py

Here we run `prepxml_SBAS.py` to create the 2 needed files:

- data.xml 
- sbas.xml

To use MinTS, we would run `prepxml_MinTS.py` to create:

- data.xml
- mints.xml
        
These files are needed by `PrepIgramStack.py`. 

We must first switch to the GIAnT folder in which `prepxml_SBAS.py` is contained, then call it. Otherwise, `prepxml_SBAS.py` will not be able to find the file 'date.mli.par', which holds necessary processing information. 

**Create a variable holding the general path to the GIAnT code base and download GIAnT from the asf-jupyter-data S3 bucket, if not present.**

GIAnT is no longer supported (Python 2). This unofficial version of GIAnT has been partially ported to Python 3 to run this notebook. Only the portions of GIAnT used in this notebook have been tested.

In [ ]:
giant_path = "/home/jovyan/.local/GIAnT/SCR"

if not Path('/home/jovyan/.local/GIAnT').exists():
    download_path = 's3://asf-jupyter-data/GIAnT_5_21.zip'
    output_path = f'/home/jovyan/.local/{Path(download_path).name}'

    !aws --region=us-east-1 --no-sign-request s3 cp $download_path $output_path
    
    if Path(output_path).is_file():        
        !unzip $output_path -d /home/jovyan/.local/
        Path(output_path).unlink()

**Run prepxml_SBAS.py and check the output to confirm that your input values are correct:**

In [ ]:
# !python $giant_path/prepxml_SBAS.py # this has already been done. data.xml and sbas.xml already exist

**Make sure the two requisite xml files (data.xml and sbas.xml) were produced after running prepxml_SBAS.py.**

**Display the contents of data.xml:**

In [ ]:
data_xml_path = work_path/'data.xml'

if data_xml_path.exists():
    !cat $data_xml_path

**Display the contents of sbas.xml:**

In [ ]:
sbas_xml_path = work_path/'sbas.xml'

if sbas_xml_path.exists():
    ! cat $sbas_xml_path

### 2.5 Create userfn.py

Before running the next piece of code, `PrepIgramStack.py`, we must create a python file called `userfn.py`. This file maps the interferogram dates to a physical file on disk. This python file must be in our working directory, `/GIAnT/SCR`. We can create this file from within the notebook using python. 

**Again, this step has already been preformed and is unnecessary, but the code is provided as an example.**

In [ ]:
# userfnTemplate = """
#!/usr/bin/env python
# import os 
"""
def makefnames(dates1, dates2, sensor):
    dirname = '{0}'
    root = os.path.join(dirname, dates1+'-'+dates2)
    #unwname = root+'_unw_phase.flt' # for potentially disruptive default values kept. 
    unwname = root+'_unw_phase_no_default.flt' # for potentially disruptive default values removed. 
    corname = root+'_corr.flt'
    return unwname, corname
"""


# with open(f'{giant_path}/userfn.py', 'w') as fid:
#     fid.write(userfnTemplate.format(work_path))

In [ ]:
print(giant_path)

## 3. Run GIAnT

We have now created all of the necessary files to run GIAnT. The full GIAnT process requires 3 function calls.

- PrepIgramStack.py
    - After PrepIgramStack.py, we will actually start running GIAnT. 
- ProcessStack.py
- SBASInvert.py
- SBASxval.py
    - This 4th function call is not necessary and we will skip it, but provides some error estimation that can be useful.
    
### 3.1 Run PrepIgramStack.py

Here we would run `PrepIgramStack.py` to create the files for GIAnT. This would read in the input data and the files we previously created and output an HDF5 file. As we do not actually need to call this, it is currently set up to display some help information.

Inputs:       
- ifg.list
- data.xml
- sbas.xml  
- interferograms
- coherence files        

Outputs:
- RAW-STACK.h5
- PNG previews under 'GIAnT/Figs/Igrams'

**Display some help information for PrepIgramStack.py:**

In [ ]:
!python $giant_path/PrepIgramStack.py -h

**Run PrepIgramStack.py (in our case, this has already been done):**

In [ ]:
# with asfn.work_dir(work_path):
#     !python $giant_path/PrepIgramStack.py

---
PrepIgramStack.py creates a file called 'RAW-STACK.h5'.

**Verify that RAW-STACK.h5 is an HDF5 file as required by the rest of GIAnT.**

In [ ]:
raw_h5 = f"{stack_path}/RAW-STACK.h5"
if not h5py.is_hdf5(raw_h5):
    print(f"Not an HDF5 file: {raw_h5}")
else:
    print(f"Confirmed: {raw_h5} is an HDF5 file.")

### 3.2 Run ProcessStack.py

This seems to be an optional step. Does atmospheric corrections and estimation of orbit residuals.

Inputs:

- HDF5 files from PrepIgramStack.py, RAW-STACK.h5
- data.xml 
- sbas.xml
- GPS Data (optional; we don't have this)
- Weather models (downloaded automatically)

Outputs: 

- HDF5 files, PROC-STACK.h5
        
These files are then fed into SBAS. 

**Display the help information for ProcessStack.py:**

In [ ]:
!python $giant_path/ProcessStack.py -h

**Run ProcessStack.py:**

In [ ]:
with asfn.work_dir(work_path):
    !python $giant_path/ProcessStack.py

---
ProcessStack.py creates a file called 'PROC-STACK.h5'.

**Verify that PROC-STACK.h5 is an HDF5 file as required by the rest of GIAnT:**

In [ ]:
proc_h5 = f"{stack_path}/PROC-STACK.h5"
if not h5py.is_hdf5(proc_h5):
    print(f"Not an HDF5 file: {proc_h5}")
else:
    print(f"Confirmed: {proc_h5} is an HDF5 file.")

### 3.3 Run SBASInvert.py

Invert the time series. 
 
Inputs

- HDF5 file, PROC-STACK.h5
- data.xml
- sbas.xml

Outputs

- HDF5 file: LS-PARAMS.h5

**Display the help information for SBASInvert.py:**

In [ ]:
!python $giant_path/SBASInvert.py -h

**Run SBASInvert.py:**

In [ ]:
with asfn.work_dir(work_path):
    !python $giant_path/SBASInvert.py

---
SBASInvert.py creates a file called 'LS-PARAMS.h5'.

**Verify that LS-PARAMS.h5 is an HDF5 file as required by the rest of GIAnT:**

In [ ]:
params_h5 = f"{stack_path}/LS-PARAMS.h5"
if not h5py.is_hdf5(params_h5):
    print(f"Not an HDF5 file: {params_h5}")
else:
    print(f"Confirmed: {params_h5} is an HDF5 file.")

### 3.4 Run SBASxval.py

Get an uncertainty estimate for each pixel and epoch using a Jacknife test. We are skipping this function as we won't be doing anything with its output and it takes a significant amount of time to run relative to the other GIAnT functions.
 
Inputs: 

- HDF5 files, PROC-STACK.h5
- data.xml
- sbas.xml

Outputs:

- HDF5 file, LS-xval.h5

**Display the help information for SBASxval.py:**

In [ ]:
#!python $giant_path/SBASxval.py -h

**Run SBASxval.py:**

In [ ]:
# with asfn.work_dir(work_path):
#     !python $giant_path/SBASxval.py

---
SBASxval.py  creates a file called 'LS-xval.h5'.

**Verify that LS-xval.h5 is an HDF5 file as required by the rest of GIAnT:**

In [ ]:
'''
xval_h5 = f"{stack_path}/LS-xval.h5"
if not h5py.is_hdf5(xval_h5):
    print(f"Not an HDF5 file: {xval_h5}")
else:
    print(f"Confirmed: {xval_h5} is an HDF5 file.")
'''

## 4. Data Visualization

Now we visualize the data. This is largely copied from Lab 4.

**Create a directory in which to store our plots and move into it:**

In [ ]:
plot_dir = work_path/'plots'

if not plot_dir.exists():
    plot_dir.mkdir()

**Load the stack produced by GIAnT and read it into an array so we can manipulate and display it:**

In [ ]:
f = h5py.File(params_h5, 'r')

**List all groups ('key's) within the HDF5 file that has been loaded into the object 'f'**

In [ ]:
print("Keys: %s" %f.keys())

Details on what each of these keys means can be found in the GIAnT documentation. For now, the only keys with which we are concerned are `recons` (the filtered time series of each pixel) and `dates` (the dates of acquisition). It is important to note that the dates are given in a type of Julian Day number called Rata Die number. This will have to be converted later, but this can easily be done via one of several different methods in Python.

**Get our data from the stack:**

In [ ]:
data_cube = f['recons'][()]

**Get the dates for each raster from the stack:**

In [ ]:
dates = list(f['dates']) # these dates appear to be given in Rata Die style: floor(Julian Day Number - 1721424.5). 
if data_cube.shape[0] is not len(dates):
    print('Problem:')
    print('Number of rasters in data_cube: ',data_cube.shape[0])
    print('Number of dates: ',len(dates))

**Plot and save amplitude image with transparency determined by alpha (SierraNegra-dBScaled-AmplitudeImage.png):**

In [ ]:
plt.rcParams.update({'font.size': 14})
radar_tiff = f"{work_path}/20161119-20170106_amp.tiff"
radar=gdal.Open(radar_tiff)
im_radar = radar.GetRasterBand(1).ReadAsArray()
radar = None
dbplot = np.ma.log10(im_radar)
vmin=np.percentile(dbplot,3)
vmax=np.percentile(dbplot,97)
fig = plt.figure(figsize=(18,10)) # Initialize figure with a size
ax1 = fig.add_subplot(111) # 221 determines: 2 rows, 2 plots, first plot
ax1.imshow(dbplot, cmap='gray',vmin=vmin,vmax=vmax,alpha=1);
plt.title('Example dB-scaled SAR Image for Ifgrm 20161119-20170106')
plt.grid()
plt.savefig(f'{plot_dir}/SierraNegra-dBScaled-AmplitudeImage.png',dpi=200,transparent='false')

**Display and save an overlay of the clipped deformation map and amplitude image (SierraNegra-DeformationComposite.png):**

In [ ]:
# We will define a short function that can plot an overaly of our radar image and deformation map. 
def defNradar_plot(deformation, radar):
    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(111)
    vmin = np.percentile(radar, 3)
    vmax = np.percentile(radar, 97)
    ax.imshow(radar, cmap='gray', vmin=vmin, vmax=vmax)
    fin_plot = ax.imshow(deformation, cmap='RdBu', vmin=-50.0, vmax=50.0, alpha=0.75)
    fig.colorbar(fin_plot, fraction=0.24, pad=0.02)
    ax.set(title="Integrated Defo [mm] Overlain on Clipped db-Scaled Amplitude Image")
    plt.grid()
    
# Get deformation map and radar image we wish to plot
deformation = data_cube[data_cube.shape[0]-1]

# Call function to plot an overlay of our deformation map and radar image.
defNradar_plot(deformation, dbplot)
plt.savefig(f'{plot_dir}/SierraNegra-DeformationComposite.png', dpi=200, transparent='false')

**Convert from Rata Die number (similar to Julian Day number) contained in 'dates' to Gregorian date:**

In [ ]:
tindex = []
for d in dates:
    tindex.append(date.fromordinal(int(d)))

**Create an animation of the deformation**

In [ ]:
%%capture
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(111)
ax.axis('off')
vmin=np.percentile(data_cube.flatten(), 5)
vmax=np.percentile(data_cube.flatten(), 95)


im = ax.imshow(data_cube[0], cmap='RdBu', vmin=-50.0, vmax=50.0)
ax.set_title("Animation of Deformation Time Series - Sierra Negra, Galapagos")
fig.colorbar(im)
plt.grid()

def animate(i):
    ax.set_title("Date: {}".format(tindex[i]))
    im.set_data(data_cube[i])
    
ani = matplotlib.animation.FuncAnimation(fig, animate, frames=data_cube.shape[0], interval=400)

**Configure matplotlib's RC settings for the animation:**

In [ ]:
rc('animation', embed_limit=10.0**9)

**Create a javascript animation of the time-series running inline in the notebook:**

In [ ]:
HTML(ani.to_jshtml())

**Save the animation as a 'gif' file (SierraNegraDeformationTS.gif):**

In [ ]:
ani.save(f'{plot_dir}/SierraNegraDeformationTS.gif', writer='pillow', fps=2)

## 5. Alter the time filter parameter

Looking at the video above, you may notice that the deformation has a very smoothed appearance. This may be because of our time filter which is currently set to 1 year ('filt=1.0' in the prepxml_SBAS.py code). Let's repeat the lab from there with 2 different time filters.

First, using no time filter ('filt=0.0') and then using a 1 month time filter ('filt=0.082'). Change the output file name for anything you want saved (e.g., 'SierraNegraDeformationTS.gif' to 'YourDesiredFileName.gif'). Otherwise, it will be overwritten.

- How did these changes affect the output time series?
- How might we figure out the right filter length?
- What does this say about the parameters we select?

## 6. Clear data (optional)

This lab has produced a large quantity of data. If you look at this notebook in your home directory, it should now be ~13 MB. This can take a long time to load in a Jupyter Notebook. It may be useful to clear the cell outputs. 

To clear the cell outputs, go Cell->All Output->Clear. This will clear the outputs of the Jupyter Notebook and restore it to its original size of ~60 kB. This will not delete any of the files we have created.

*GEOS 657-Lab9-InSARTimeSeriesAnalysis.ipynb - Version 1.3.3 - November 2021*

*Version Changes:*

- *asf_notebook -> opensarlab_lib*
- *html -> markdown*